# Switzerland and Black Slavery

## Header 
Author : Amina Matt and Yichen Wang  
Date created : 09.12.2021  
Date last modified : 22.12.2021  
Python version : 3.8  
Description : Enriching original dataset from [CARICOM Compilation Archive (CCA)](https://louverture.ch/cca/) with geographical information (coordinates).
The goal is to obtain 
- origin coordinates
- colonial location coordinates

for each entry.

## Librairies

In [635]:
# -*- coding: utf-8 -*-

import geonamescache # Retrieve names, ISO codes of continents, countries as well as US states and counties as Python dictionaries
import json #for dumpint of dictionary into json files
import pandas as pd #Pandas provides data structure and data analysis tools
from pandas.io.json import json_normalize
import re 

## Initialization

In [636]:
#PATHS
DATA_FOLDER = './data/'
caricom_sample = DATA_FOLDER +'Caricom_Archive_Sample_Schema1.txt' #sample text for testing 
caricom = DATA_FOLDER +'Caricom_Archive.txt'

# Add geographic coordinates 

# Load 

In [637]:
a_file = open("from_caricom.json", "r")
jsonList = json.load(a_file)

In [638]:
#jsonList

## From JSONs to Dataframe

In [639]:
#transform JSON list into a dataframe 
caricomDataRaw = pd.json_normalize(jsonList)

In [640]:
tot = len(caricomDataRaw['origin'])
filled = len(caricomDataRaw[caricomDataRaw['origin'].isnull()])
print(f'We have {tot} entries, and {filled} empty entries')

We have 347 entries, and 127 empty entries


In [646]:
caricomDataRaw.sample(6)

,person,date,origin,colonial_Location,confidence_date,confidence_person,confidence_origin,activities,whole_entry
288,Hans Pieter Anthonij,None,Biel,East Indies,0,16.67,20.00,None,=> The following Swiss are mentioned in 1727 i...
240,John Bartow Prevost,None,Geneva,Louisiana,0,20.00,100.00,"[slave owner, plantation owner]","=> John Bartow Prevost (1766–1825), whose fath..."
318,John Jacob Flournoy,None,None,Switzerland,0,9.09,100.00,"[slave owner, racist]",=> John Jacob Flournoy (1808–1879) from a fami...
317,Marc Warnery,1833,Morges,Switzerland,25.00,100.00,100.00,"[slave owner, military, plantation owner]","=> In 1833, Marc Warnery (1797-1836) from Morg..."
220,Mary Elizabeth DeSaussure,1747,Lausanne,Carolinas,7.69,9.09,100.00,"[slave owner, plantation owner]","=> Mary Elizabeth DeSaussure (1747 – 1823), da..."
118,Judith Coin,1770,Echallens,Antigua and Barbuda,50.00,50.00,100.00,None,"=> In 1770, Judith Coin from Echallens in the ..."


In [643]:
caricomDataRaw['person'] = caricomDataRaw['person'].apply(lambda x: re.sub('[^A-Za-z ]+', '', x))

## Geolocalisation

### Load

In [586]:
# countries
gc = geonamescache.GeonamesCache()
countries = gc.get_countries_by_names()
# cities
# For cities we don't need to load a set we can directly use gc.get_cities_by_name(name) function

### Geoinformation for colonial location 
Colonial locations are mostly countries, and some regions (e.g. East Indies)

In [587]:
countries_raw = caricomDataRaw['colonial_Location'].unique()

In [588]:
countries_raw

array(['Antigua and Barbuda', 'Barbados', 'Dominica', 'Grenada', 'Guyana',
       'Haiti', 'Jamaica', 'Montserrat', 'Cuba', 'Netherlands Antilles',
       'French West Indies', 'Danish West Indies', 'Venezuela',
       'Bermudas', 'Alabama', 'Arkansas', 'California', 'Carolinas',
       'Florida', 'Georgia', 'Louisiana', 'Mississippi', 'Brazil',
       'Southern Africa', 'East Indies', 'Switzerland'], dtype=object)

#### United States
We need to treat the us states separately

In [589]:
us_states = gc.get_us_states()
#us_states

In [590]:
us_states_dic = []
for i in range(len(list(us_states.keys())[0:51])):
    key = list(us_states.keys())[i]
    #print(key)
    all_inf = us_states[str(key)]
    name = all_inf['name']
    us_states_dic.append([name,key])
    #print(name)
#us_states_dic

To identify a precise location for each states we load capitals.

In [591]:
### capital 
capital_dic={
    'Alabama': 'Montgomery',
    'Alaska': 'Juneau',
    'Arizona':'Phoenix',
    'Arkansas':'Little Rock',
    'California': 'Sacramento',
    'Colorado':'Denver',
    'Connecticut':'Hartford',
    'Delaware':'Dover',
    'Florida': 'Tallahassee',
    'Georgia': 'Atlanta',
    'Hawaii': 'Honolulu',
    'Idaho': 'Boise',
    'Illinios': 'Springfield',
    'Indiana': 'Indianapolis',
    'Iowa': 'Des Monies',
    'Kansas': 'Topeka',
    'Kentucky': 'Frankfort',
    'Louisiana': 'Baton Rouge',
    'Maine': 'Augusta',
    'Maryland': 'Annapolis',
    'Massachusetts': 'Boston',
    'Michigan': 'Lansing',
    'Minnesota': 'St. Paul',
    'Mississippi': 'Jackson',
    'Missouri': 'Jefferson City',
    'Montana': 'Helena',
    'Nebraska': 'Lincoln',
    'Neveda': 'Carson City',
    'New Hampshire': 'Concord',
    'New Jersey': 'Trenton',
    'New Mexico': 'Santa Fe',
    'New York': 'Albany',
    'North Carolina': 'Raleigh',
    'North Dakota': 'Bismarck',
    'Ohio': 'Columbus',
    'Oklahoma': 'Oklahoma City',
    'Oregon': 'Salem',
    'Pennsylvania': 'Harrisburg',
    'Rhoda Island': 'Providence',
    'South Carolina': 'Columbia',
    'South Dakoda': 'Pierre',
    'Tennessee': 'Nashville',
    'Texas': 'Austin',
    'Utah': 'Salt Lake City',
    'Vermont': 'Montpelier',
    'Virginia': 'Richmond',
    'Washington': 'Olympia',
    'West Virginia': 'Charleston',
    'Wisconsin': 'Madison',
    'Wyoming': 'Cheyenne'  
}


In [592]:
countries_raw

array(['Antigua and Barbuda', 'Barbados', 'Dominica', 'Grenada', 'Guyana',
       'Haiti', 'Jamaica', 'Montserrat', 'Cuba', 'Netherlands Antilles',
       'French West Indies', 'Danish West Indies', 'Venezuela',
       'Bermudas', 'Alabama', 'Arkansas', 'California', 'Carolinas',
       'Florida', 'Georgia', 'Louisiana', 'Mississippi', 'Brazil',
       'Southern Africa', 'East Indies', 'Switzerland'], dtype=object)

## List with countries information for colonial locations 

In [593]:
#states list
only_names = [x[0] for x in us_states_dic]


#Create a dictionary with geo information of countries of interest
countries_geo_dict = []
for countryName in countries_raw:
     
    try : 
        country = countries[countryName]
        countries_geo_dict.append(country)
    except :
        print('This is not a country '+str(countryName))
        country = None
        countries_geo_dict.append(country)
    #For US States
    try :
        index = only_names.index(countryName) 
        countryInitial = us_states_dic[index][1]
        country = us_states[countryInitial]
        countries_geo_dict.append(country)
    except :
        continue

countries_geo_dict.append(country)

This is not a country French West Indies
This is not a country Danish West Indies
This is not a country Bermudas
This is not a country Alabama
This is not a country Arkansas
This is not a country California
This is not a country Carolinas
This is not a country Florida
This is not a country Louisiana
This is not a country Mississippi
This is not a country Southern Africa
This is not a country East Indies


#### REGIONAL LOCATIONS
For the entries that are not US States neither countries we map them to a greographical location 

In [594]:

# We have 5 entries in the dataframe that must be mapped to another geoname
# --French West Indies-- for now we will map this to Guadeloupe
country = countries['Guadeloupe']
countries_geo_dict.append(country)
# --Danish West Indies-- 
country = countries['U.S. Virgin Islands']
countries_geo_dict.append(country)
# --Southern Africa-- for now we will map to South Africa
country = countries['South Africa']
countries_geo_dict.append(country)
# East Indies' is maped to Indonesia
country = countries['Indonesia']
countries_geo_dict.append(country)
#Bermudas is Bermuda 
country = countries['Bermuda']
countries_geo_dict.append(country)

In [595]:
len(countries_geo_dict)

39

In [596]:
#countries_clean
#to remove None values in list
countries_geo_dict_final = []
for val in countries_geo_dict:
    if val != None :
        countries_geo_dict_final.append(val)       

In [597]:
#transform JSON list into a dataframe 
colonial_countries = pd.json_normalize(countries_geo_dict_final)
colonial_countries = colonial_countries.drop_duplicates(subset=['name'],keep='last')
colonial_countries.head()

,geonameid,name,iso,iso3,isonumeric,fips,continentcode,capital,areakm2,population,tld,currencycode,currencyname,phone,postalcoderegex,languages,neighbours,code
0,3576396,Antigua and Barbuda,AG,ATG,28.0,AC,NA,St. John's,443.0,86754.0,.ag,XCD,Dollar,+1-268,,en-AG,,NaN
1,3374084,Barbados,BB,BRB,52.0,BB,NA,Bridgetown,431.0,285653.0,.bb,BBD,Dollar,+1-246,^(?:BB)*(\d{5})$,en-BB,,NaN
2,3575830,Dominica,DM,DMA,212.0,DO,NA,Roseau,754.0,72813.0,.dm,XCD,Dollar,+1-767,,en-DM,,NaN
3,3580239,Grenada,GD,GRD,308.0,GJ,NA,St. George's,344.0,107818.0,.gd,XCD,Dollar,+1-473,,en-GD,,NaN
4,3378535,Guyana,GY,GUY,328.0,GY,SA,Georgetown,214970.0,748486.0,.gy,GYD,Dollar,592,,en-GY,"SR,BR,VE",NaN


In [598]:
#add capitals for us states
for name in colonial_countries['name'] :
    if name in only_names: #only names is the list of US states
        print(f'We have a us state: {name}')
        capital = capital_dic[name]
        print(f'The capital is {capital}')
        #df = colonial_countries.loc[colonial_countries['name'] == name]
        colonial_countries.loc[(colonial_countries['name'] == name),'capital']=capital
        #colonial_countries.insert(2,'capital', capital)
colonial_countries.iloc[15:20]

We have a us state: Alabama
The capital is Montgomery
We have a us state: Arkansas
The capital is Little Rock
We have a us state: California
The capital is Sacramento
We have a us state: Florida
The capital is Tallahassee
We have a us state: Georgia
The capital is Atlanta
We have a us state: Louisiana
The capital is Baton Rouge
We have a us state: Mississippi
The capital is Jackson


,geonameid,name,iso,iso3,isonumeric,fips,continentcode,capital,areakm2,population,tld,currencycode,currencyname,phone,postalcoderegex,languages,neighbours,code
16,4197000,Georgia,NaN,NaN,NaN,13,NaN,Atlanta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GA
17,4331987,Louisiana,NaN,NaN,NaN,22,NaN,Baton Rouge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA
18,4436296,Mississippi,NaN,NaN,NaN,28,NaN,Jackson,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MS
19,3469034,Brazil,BR,BRA,76.0,BR,SA,Brasilia,8511965.0,201103330.0,.br,BRL,Real,55,^(\d{8})$,"pt-BR,es,en,fr","SR,PE,BO,UY,GY,PY,GF,VE,CO,AR",NaN
21,2658434,Switzerland,CH,CHE,756.0,SZ,EU,Berne,41290.0,7581000.0,.ch,CHF,Franc,41,^(\d{4})$,"de-CH,fr-CH,it-CH,rm","DE,IT,LI,FR,AT",NaN


We now have a dataframe colonial_countries that contains information about countries we identified in the colonial location columns from dataset

##### Final dataframe a geoname id

In [599]:
colonial_countries['name']

0      Antigua and Barbuda
1                 Barbados
2                 Dominica
3                  Grenada
4                   Guyana
5                    Haiti
6                  Jamaica
7               Montserrat
8                     Cuba
9     Netherlands Antilles
10               Venezuela
11                 Alabama
12                Arkansas
13              California
14                 Florida
16                 Georgia
17               Louisiana
18             Mississippi
19                  Brazil
21             Switzerland
22              Guadeloupe
23     U.S. Virgin Islands
24            South Africa
25               Indonesia
26                 Bermuda
Name: name, dtype: object

We now have the countries  information about colonial location to the initial dataframe (Note that it doesnt have the capitals coordinates yey)
The merge is done with the initial dataframe, however as notices some "colonial_location" are exactly countries.For the merge to work we do a mapping.
There are 5 entries in the dataframe that must be mapped to another geoname

Note that we prepare the necessary information in the geographical dataframe, i.e. the 5 mappings have a correspondance.
i.e.  
colonial_location -> tmp colonial_location   
East Indies       -> Indonesia  

In [600]:
def to_tmp_colonial(colonial_Location):
    '''
    Input:location from the original dataframe
    Output: location that has a geoinformation 
    Requirements: - 
    Description: Map colonial location with the map list defined
    '''
    listToModify = ['French West Indies','Danish West Indies','Southern Africa', 'East Indies','Bermudas'] # colonial_location column in intiial dataset 
    listReplacement= ['Guadeloupe','U.S. Virgin Islands','South Africa','Indonesia','Bermuda']           # country column in geographic information dataset
    
    if colonial_Location in listToModify:
        index = listToModify.index(colonial_Location)
        colonial_Location =  listReplacement[index]
        
    return colonial_Location 

In [601]:
caricomDataRaw['tmp_colonial'] = caricomDataRaw['colonial_Location'] .apply(lambda x : to_tmp_colonial(x)) 

In [602]:
caricomDataRaw.iloc[190:200]

,person,date,origin,colonial_Location,confidence_date,confidence_person,confidence_origin,activities,whole_entry,tmp_colonial
190,A Allegrin,citizenship,None,Danish West Indies,25.000000,25.00,0.00,None,=> According to the Royal Danish State Archive...,U.S. Virgin Islands
191,Urbain Roger,1757,None,Danish West Indies,12.500000,20.00,33.33,[slave owner],"=> In 1757, Urbain Roger (1726–1791) from a Hu...",U.S. Virgin Islands
192,Reinhard Iselin,1744-1825,Zurich,Danish West Indies,14.285714,11.11,50.00,"[slave owner, trading]","=> In the 1770s, Reinhard Iselin motivated Sal...",U.S. Virgin Islands
193,Hans Conrad von Orelli,1805,Zurich,Danish West Indies,100.000000,100.00,0.00,[trading],"=> In 1805, Hans Conrad von Orelli, merchant f...",U.S. Virgin Islands
194,Bethany,None,,Danish West Indies,9.090909,14.29,0.00,"[slave owner, plantation owner, trading]",=> The Moravian missionaries settled in the Da...,U.S. Virgin Islands
195,Danes,1733,Martinique,Danish West Indies,100.000000,100.00,0.00,"[slave owner, military, plantation owner]","=> In 1733, approximately 150 Akwamu slaves st...",U.S. Virgin Islands
196,Robert Lutz,1842,None,Danish West Indies,33.333333,50.00,0.00,"[slave owner, trading]","=> In 1842, Swiss merchant Robert Lutz (1823-1...",U.S. Virgin Islands
197,Hieronymus Sailer,1528,St. Gallen,Venezuela,25.000000,33.33,100.00,"[slave owner, trading]","=> In 1528, Hieronymus Sailer (1495-1559) from...",Venezuela
198,Friedrich Ludwig,1779-1831,None,Bermudas,11.111111,33.33,50.00,[military],=> Friedrich Ludwig Im Thurn‏‎ (1779-1831) fro...,Bermuda
199,Ott,None,Guttannen,Alabama,100.000000,100.00,0.00,[slave owner],=> Members of the Ott family from Guttannen (B...,Alabama


## Add capitals

We need to add capitals as we are interested in point locations, an take care of diambuigations.

In [603]:
from geonamescache.mappers import country
mapper = country(from_key='name', to_key='iso')

def getCapital_coo(country,capital_Name):
    '''
    Input: recognized country name and recognized capital name
    Output: geographical coordinates for capitals
    Requirements: country from geonamescache.mapperslibrary  
    Description: find the coordinates of countries 
    '''
    #known disparities 
    if capital_Name == 'Berne':
        capital_Name = 'Bern'
    if capital_Name == 'Brasilia':
        capital_Name = 'Brasília' 
    if capital_Name == 'St. George\'s':    
        capital_Name = 'Saint George\'s'
        
    lat = 'None'
    lon = 'None'
    all_info = gc.search_cities(capital_Name)
    countrycode = mapper(country) #get country code
    
    #default 
    origin_geo = all_info[0] 
    
    #we check if the country code is similar to avoid ambiguities
    for sub in all_info :
        if sub['countrycode'] == countrycode:
            origin_geo = sub
   
    if len(origin_geo)>0:
        lat = origin_geo['latitude']
        lon = origin_geo['longitude']
        
    return lat,lon

In [604]:
#Example
getCapital_coo('Bermuda','Hamilton')

(32.2949, -64.78303)

In [605]:
colonial_countries['latitude'] = colonial_countries.apply(lambda x : getCapital_coo(x['name'],x['capital'])[0],axis=1)
colonial_countries['longitude'] = colonial_countries.apply(lambda x : getCapital_coo(x['name'],x['capital'])[1],axis=1)
#colonial_countries

## Merge 
We can now merge the geographic informations with the initial dataset

In [606]:
#prepare the colonial countries dataframes to be added 
colonial_countries_for_merge = colonial_countries[['geonameid','name','capital','latitude','longitude']]
colonial_countries_for_merge = colonial_countries_for_merge.rename(columns={"geonameid": "col_loc_geonameid",
                                                                            "name": "col_loc_geo_name",
                                                                            'capital':'col_capital',
                                                                            'latitude':"col_latitude",
                                                                            'longitude':"col_longitude"})

In [607]:
caricom_col_loc = caricomDataRaw.merge(colonial_countries_for_merge, left_on='tmp_colonial', right_on='col_loc_geo_name')

In [608]:
## Dataframe with geographic information for colonial locations

In [609]:
caricom_col_loc.head()

,person,date,origin,colonial_Location,confidence_date,confidence_person,confidence_origin,activities,whole_entry,tmp_colonial,col_loc_geonameid,col_loc_geo_name,col_capital,col_latitude,col_longitude
0,Arthur Thellusson,None,Geneva,Antigua and Barbuda,100.000000,25.00,100.00,[slave owner],"=> Arthur Thellusson, son of Lord Rendlesham a...",Antigua and Barbuda,3576396,Antigua and Barbuda,St. John's,17.12096,-61.84329
1,Jamaica Kincaid,None,None,Antigua and Barbuda,100.000000,100.00,0.00,None,"=> In her book A Small Place (1988), the Antig...",Antigua and Barbuda,3576396,Antigua and Barbuda,St. John's,17.12096,-61.84329
2,Hans Conrad Hottinger,Hottinguer,Zurich,Antigua and Barbuda,14.285714,25.00,100.00,[plantation owner],=> Hans Conrad Hottinger (Hottinguer) (1764–18...,Antigua and Barbuda,3576396,Antigua and Barbuda,St. John's,17.12096,-61.84329
3,Hans Conrad Hottinger,1759­–1839,Saint-Aubin,Antigua and Barbuda,9.090909,25.00,33.33,None,=> Hans Conrad Hottinger’s business partner wa...,Antigua and Barbuda,3576396,Antigua and Barbuda,St. John's,17.12096,-61.84329
4,David de Duveluz,None,Bournens,Antigua and Barbuda,100.000000,33.33,0.00,[plantation owner],"=> David de Duveluz, from Bournens in the Cant...",Antigua and Barbuda,3576396,Antigua and Barbuda,St. John's,17.12096,-61.84329


In [610]:
print(f'We now have {len(caricom_col_loc)} entries')

We now have 327 entries


### Geoinformation for location of origin 
In the origin location we have mostly cities and region, if the specific places doesn't correspond to any know cities we use Switzerland as a general indicator.

In [611]:
origin_locs = caricom_col_loc['origin'].unique()
origin_locs

array(['Geneva', None, 'Zurich', 'Saint-Aubin', 'Bournens', 'Bourmens',
       'St. Gallen', 'Echallens', 'Schaffhausen', 'Obersimmental', 'Bâle',
       '', 'Noraz', 'Le Locle', 'Rehetobel', 'Brazil', 'Morges', 'Basel',
       'Ropraz', 'Gourgy', 'Africa', 'Neuchâtel', 'Vevey', 'Lausanne',
       'Lelienburg', 'Bürglen', 'Burgdorf', 'Thurgau', 'Treytorrens',
       'Speicher', 'Walenstadt', 'Aarau', 'La Tour-de-Peilz', 'Lutry',
       'Murten', 'Switzerland', 'La Rochelle', 'Versoix', 'Sonvillier',
       'Schftland', 'Saint-Domingue', 'Berne', 'Hunziker', 'Solothurn',
       'Aargau', 'Dornach', 'Lucerne', 'Graubünden', 'Jamaica',
       'Rougement', 'Yverdon', 'Mtier', 'Winterthur', 'Biel',
       'Bischofszell', 'Unterseen BE', 'Couvet', 'Nantes', 'Zofingen',
       'Klosters', 'Fribourg', 'Saint-Saphorin', 'Saint-Lgier-La Chisaz',
       'Saint-Sulpice', 'La Cluse', 'Schwyz', 'Vendlincourt', 'Lenzburg',
       'Chur', 'Avenches', 'Martinique', 'Guttannen', 'North Carolina',
      

## Geographical information for Swiss cities

In [612]:
#counters
found = 0 
no_correspondance = 0 
origin_geo_dic = []
#loop to fill with geo information for origin countries
for origin in origin_locs:
        #sanity check for empty entry
        if (origin != None) and (origin != ''):
            origin_geo_all = gc.search_cities(origin)

            if len(origin_geo_all) > 0 :
                dict_geo = origin_geo_all[0]
                if(dict_geo['countrycode']=='CH'):
                    dict_geo['origin_as_found']=origin
                    #print(dict_geo)
                    found = found +1 
                    origin_geo_dic.append(dict_geo)
                    #print(f'There is a correspondance for {origin}')


print(f'There are {found} found')

There are 18 found


In [613]:
#origin_geo_dic

In [614]:
#transform JSON list into a dataframe 
origin_cities = pd.json_normalize(origin_geo_dic)
origin_cities = origin_cities[['origin_as_found','geonameid','name','latitude','longitude']]

In [615]:
origin_cities = origin_cities.drop_duplicates()
origin_cities = origin_cities.rename(columns={"geonameid": "origin_loc_geonameid", "name": "origin_geo_name",'latitude':"origin_latitude",'longitude':"origin_longitude"})

#### Geodata for origin locations final dataframe

In [616]:
origin_cities

,origin_as_found,origin_loc_geonameid,origin_geo_name,origin_latitude,origin_longitude
0,Geneva,2660646,Genève,46.20222,6.14569
1,Zurich,2657896,Zürich,47.36667,8.55000
2,St. Gallen,2658822,Sankt Gallen,47.42391,9.37477
3,Schaffhausen,2658761,Schaffhausen,47.69732,8.63493
4,Basel,2661604,Basel,47.55839,7.57327
6,Neuchâtel,2659496,Neuchâtel,46.99179,6.93100
7,Vevey,2658145,Vevey,46.46299,6.84345
8,Lausanne,2659994,Lausanne,46.51600,6.63282
9,Aarau,2661881,Aarau,47.39254,8.04422
10,Berne,2661552,Bern,46.94809,7.44744


## Merge between geographical information for Swiss cities and origin locations

In [617]:
len(caricom_col_loc)

327

In [618]:
caricom_both_loc = caricom_col_loc.merge( origin_cities,how = 'outer', left_on='origin', right_on='origin_as_found')

In [619]:
len(caricom_both_loc)

327

In [620]:
origin_geo_all = gc.get_cities_by_name('Bern')[0]
keyId_bern = list(origin_geo_all.keys())[0]

In [621]:
import numpy as np 
caricom_both_loc['origin_geo_name'] = caricom_both_loc['origin_geo_name'].replace(np.nan, 'Bern')
caricom_both_loc['origin_loc_geonameid'] = caricom_both_loc['origin_loc_geonameid'].replace(np.nan,keyId_bern)

In [622]:
#this shows why we need to merge on the origin as found
#caricom_both_loc[caricom_both_loc['origin_as_found']=='Biel']

In [623]:
caricom_both_loc[['person','date','origin','colonial_Location','origin_as_found',
                  'col_loc_geonameid','col_loc_geo_name','col_capital','col_latitude',
                 'col_longitude','origin_loc_geonameid','origin_geo_name',
                  'origin_latitude','origin_longitude']]

,person,date,origin,colonial_Location,origin_as_found,col_loc_geonameid,col_loc_geo_name,col_capital,col_latitude,col_longitude,origin_loc_geonameid,origin_geo_name,origin_latitude,origin_longitude
0,Arthur Thellusson,None,Geneva,Antigua and Barbuda,Geneva,3576396,Antigua and Barbuda,St. John's,17.12096,-61.84329,2.66065e+06,Genève,46.20222,6.14569
1,Peter Thelluson,1767,Geneva,Barbados,Geneva,3374084,Barbados,Bridgetown,13.10732,-59.62021,2.66065e+06,Genève,46.20222,6.14569
2,JeanAntoine Bertrand,None,Geneva,Dominica,Geneva,3575830,Dominica,Roseau,15.30174,-61.38808,2.66065e+06,Genève,46.20222,6.14569
3,Peter Thelluson,1768,Geneva,Grenada,Geneva,3580239,Grenada,St. George's,12.05288,-61.75226,2.66065e+06,Genève,46.20222,6.14569
4,Jan Vincent Mittelholzer,1760,Geneva,Guyana,Geneva,3378535,Guyana,Georgetown,6.80448,-58.15527,2.66065e+06,Genève,46.20222,6.14569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
322,Carl Vogt,None,Germany,Switzerland,NaN,2658434,Switzerland,Berne,46.94809,7.44744,2661552,Bern,NaN,NaN
323,Adolf Guyer,1860,Neuthal,Switzerland,NaN,2658434,Switzerland,Berne,46.94809,7.44744,2661552,Bern,NaN,NaN
324,Henry Hotze,1833–1887,Rümlang,Switzerland,NaN,2658434,Switzerland,Berne,46.94809,7.44744,2661552,Bern,NaN,NaN
325,Jost Brgi,1552-1632,Lichtensteig,Switzerland,NaN,2658434,Switzerland,Berne,46.94809,7.44744,2661552,Bern,NaN,NaN


## Dump DF 

In [624]:
caricom_both_loc.to_pickle("./caricom_with_geoid.pkl")

## Read pickle

In [625]:
unpickled_df = pd.read_pickle("./caricom_with_geoid.pkl")